In [19]:
%matplotlib inline
import numpy as np
import pandas as pd
import networkx as nx
from networks import DiscourseNetworks

# Reading dna-party dataset

In [12]:
# This dataset has been created by Caroline Bhattacharya
data = pd.read_csv('../dataset/dna-party-dataset.csv', sep=",")

# Rows in dataset
data

,statement ID,time,document ID,document type,text,mp,party,concept,agreement,category
0,480,2010-05-07,2,EoV,"Mir ist bewusst, dass es inzwischen unausweich...","Aschenberg-Dugnus, Christine",FDP,Aid_Currency stabilisation,1,Policy
1,479,2010-05-07,2,EoV,Die von der Bundesregierung nun eingeleiteten ...,"Aschenberg-Dugnus, Christine",FDP,Crisis_Contagion,1,Polity
2,2,2010-05-07,2,EoV,Die von der Bundesregierung nun eingeleiteten ...,"Aschenberg-Dugnus, Christine",FDP,Alternatives,0,Politics
3,482,2010-05-07,2,EoV,"Gleichwohl ist daran zu erinnern, dass zunäch...","Aschenberg-Dugnus, Christine",FDP,EMU_Self-responsibility,1,Polity
4,481,2010-05-07,2,EoV,Die Zugehörigkeit zur Euro-Zone darf keinen A...,"Aschenberg-Dugnus, Christine",FDP,Solidarity_European,0,Polity
...,...,...,...,...,...,...,...,...,...,...
9043,4564,2015-08-19,410,EoV,Die Syriza-Regierung muss sechs der von ihr ei...,Left,Left,Aid_National sovereignty,0,Policy
9044,4567,2015-08-19,410,EoV,Eine Rückkehr zum früheren kollektiven Tarif...,Left,Left,Greece_Collective bargaining,1,Policy
9045,4574,2015-08-19,410,EoV,"Von den Maßnahmen, die die neue griechische Re...",Left,Left,Greece_Reforms_Social,1,Policy
9046,4576,2015-08-19,410,EoV,"Von den Maßnahmen, die die neue griechische Re...",Left,Left,Greece_Public sector_Job cuts,0,Policy


# Filtering dataset

In [20]:
def select_data(data, party=['CDU'], debate_date='2010-05-07', doc_type=None):
    
    # Selecting party specific data
    data = data[data['party'].isin(party)]
    
    # Selecting document type
    if doc_type in ['Speech', 'EoV']:
        data = data[data['document type'] == doc_type]
    
    # Selecting debate     
    data = data[data['time'] == debate_date]
    
    return data

In [14]:
party_data = select_data(data, party=['CDU', 'CSU'], doc_type='None', debate_date='2015-08-19')

# Computing contestedness score

In [21]:
def prepare_agreement_disagreement_data(data):
    
    concepts = {}
    
    for index, row in data.iterrows(): 
        if not row['concept'] in concepts:
            concepts[row['concept']] = [0, 0]
        concepts[row['concept']][row['agreement']] += 1
        
    return concepts

def get_contested_concepts(concept_dict):
    concept_contested_dict = {}
    
    for k, v in concept_dict.items():
        if v[0] > 0 and v[1] > 0:
            concept_contested_dict[k] = v
            
    return concept_contested_dict

def get_max_agree_disagree(data_dict):
    data = pd.DataFrame(data_dict).T
    return np.max(data[0].to_numpy() + data[1].to_numpy()), np.max([data[0].to_numpy(), data[1].to_numpy()])

def compute_concept_index(data_dict, beta=0.5, max_values=None):
    
    if max_values is None:
        max_values, min_max_values = get_max_agree_disagree(data_dict)
        print(max_values, min_max_values)
    
    for k, values in data_dict.items():
        freq = sum(values) / max_values
        closeness = 1 - abs(values[0] - values[1]) / sum(values)
        max_v = max(values)
        min_v = min(values)
#         score = beta * freq + (1 - beta) * closeness
        score = freq * closeness * (min_v / min_max_values)
        
        data_dict[k].append(score)
        
    data_list = list(data_dict.items())
    data_list = sorted(data_list, key=lambda x: x[-1][-1], reverse=True)
        
    return data_list

def show_concept_index(data_list, k=20): # Change number of displayed rows
    
    for i,v in enumerate(data_list):
        print(v)
        if i+1 == k:
            break

In [22]:
cdata_dict = prepare_agreement_disagreement_data(party_data)
cdata_dict = get_contested_concepts(cdata_dict)
print(cdata_dict)
agree_disagree_all = get_max_agree_disagree(cdata_dict)
print(agree_disagree_all)
cdata_list = compute_concept_index(cdata_dict)
show_concept_index(cdata_list)

{'Crisis management_Greek government': [14, 3], 'Greece_Political will': [14, 9], 'Fiscal policy_Investments_Greece': [1, 3], 'Aid_Growth': [1, 4], 'Greece_Debt relief': [9, 1], 'EU_Transfer union': [2, 1], 'EU_Common currency_Robustness': [1, 2], 'Aid_Success': [2, 9], 'Party politics_Previous Greek governments': [2, 1], 'Greece_Privatisation': [1, 7], 'Bundestag_Information': [1, 5], 'Aid_Private creditors': [1, 2], 'Aid_Social fairness': [1, 2], 'Fiscal policy_Bank bailouts': [1, 5], 'Solidarity_European': [1, 1], 'EU_Unity': [1, 1], 'Alternatives_Grexit': [3, 5], 'Greece_Labour market deregulation': [1, 5], 'Aid_Crisis solution/prevention': [1, 2], 'Aid_Feasibility': [1, 1], 'Greece_Debt restructuring': [2, 2]}
(23, 14)
23 14
('Greece_Political will', [14, 9, 0.5031055900621119])
('Alternatives_Grexit', [3, 5, 0.05590062111801242])
('Crisis management_Greek government', [14, 3, 0.0559006211180124])
('Aid_Success', [2, 9, 0.02484472049689441])
('Greece_Debt restructuring', [2, 2, 0.